In [ ]:
import pickle
import requests
import numpy as np
import time
import googlemaps
import json
import os
from tqdm.notebook import tqdm

In [ ]:
with open(os.path.join("processed_data", "departments.pkl"), "rb") as f:
    departments = pickle.load(f)

with open(os.path.join("processed_data", "connectivity.pkl"), "rb") as f:
    connectivity = pickle.load(f)
    
codes_to_exclude = ["2A", "2B"] #Corsica
dep_codes = sorted(departments.keys())
dep_codes = [d for d in dep_codes if d not in codes_to_exclude and len(d)<3]

cities = [departments[c]["capital"] for c in dep_codes]
cities_coords = [departments[c]["capital_coords"] for c in dep_codes]
coords_list = []
for city in cities_coords:
    coords_list.append([city["lat"], city["lng"]])

In [ ]:
google_key = "ADD_API_KEY"
gmaps = googlemaps.Client(key=google_key)

In [ ]:
td_data = {}
for i, code in enumerate(dep_codes):
    time.sleep(0.03)
    neighbor_idxs = [dep_codes.index(c) for c in connectivity[code]]
    neighbors = [coords_list[k] for k in neighbor_idxs]
    result = gmaps.distance_matrix([coords_list[i]], neighbors, mode='driving')
    connections = []
    for k, element in enumerate(result['rows'][0]["elements"]):
        neighbor = {"code": dep_codes[neighbor_idxs[k]],
                   "time": element["duration"]["value"],
                    "distance": element["distance"]["value"]
                   }
        connections.append(neighbor)
    td_data[code] = connections


In [ ]:
td_data.keys()

In [ ]:
with open("distances_times.pkl", "wb") as f:
    pickle.dump(td_data, f)
with open("distances_times.json", "w") as f:
    json.dump(td_data, f, sort_keys=True, indent=4)